In [1]:
import torch
import torch.nn as nn
import import_ipynb
from torch.nn import functional as F
from collections import OrderedDict
from CRNet import CRNet

C:\Users\ASUS\anaconda3\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\ASUS\anaconda3\lib\site-packages\numpy\.libs\libopenblas.NOIJJG62EMASZI6NYURL6JBKM4EVBGM7.gfortran-win_amd64.dll
C:\Users\ASUS\anaconda3\lib\site-packages\numpy\.libs\libopenblas.XWYDX2IKJW2NMTWSFYNGFUWKQU3LYTCZ.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [2]:
class AstrousConv(nn.Module):
    def __init__(self, in_channels, out_channels, dilation):
        super(AstrousConv, self).__init__()
    
        self.astrous_conv = nn.Sequential(
            nn.Conv2d(
                in_channels=in_channels, 
                out_channels=out_channels, 
                kernel_size=3,
                padding=dilation,
                dilation=dilation,
                bias=False
                ),
            nn.BatchNorm2d(out_channels),
            nn.ReLU()
        )
  
    def forward(self, x):
        return self.astrous_conv(x)

In [3]:
class SpatialPooling(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(SpatialPooling, self).__init__()
    
        self.global_average_pooling = nn.Sequential(
            nn.AdaptiveAvgPool2d(1),
            nn.Conv2d(
                in_channels=in_channels,
                out_channels=out_channels,
                kernel_size=1,
                bias=False
            ),
            nn.BatchNorm2d(out_channels),
            nn.ReLU()
        )
 
    def forward(self, x):
        size = x.shape[-2:]
        x = self.global_average_pooling(x)
        return F.interpolate(x, size=size, mode='bilinear', align_corners=False)

In [4]:
class ASPP(nn.Module):
    def __init__(self, in_channels=128, out_channels=256, atrous_rates=[6, 12, 18]):
        super(ASPP, self).__init__()
    
        self.atrous_convs = nn.ModuleList()
        self.atrous_convs.append(nn.Sequential(
            nn.Conv2d(
                in_channels=in_channels,
                out_channels=out_channels,
                kernel_size=1,
                bias=False
            ),
            nn.BatchNorm2d(out_channels),
            nn.ReLU()
        ))
        for rate in atrous_rates:
            self.atrous_convs.append(AstrousConv(
            in_channels=in_channels,
            out_channels=out_channels,
            dilation=rate
            ))
    
        self.spatial_pooling = SpatialPooling(
            in_channels=in_channels,
            out_channels=out_channels
        )

        self.pyramid_pooling = nn.Sequential(
            nn.Conv2d(
                in_channels=(len(self.atrous_convs) + 1) * out_channels,
                out_channels=out_channels,
                kernel_size=1,
                bias=False
            ),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(),
            nn.Dropout(p=0.5)
        )

    def forward(self, x):
        aspp_elements = []
        for atrous_conv in self.atrous_convs:
            aspp_elements.append(atrous_conv(x))
        aspp_elements.append(self.spatial_pooling(x))
        assp_stack = torch.cat(aspp_elements, dim=1)
        return self.pyramid_pooling(assp_stack)

In [5]:
def test_aspp():
    x = torch.randn((3, 128, 27, 27))
    model = ASPP()
    pred = model(x)
    print(pred.shape)

In [6]:
class CRASPPNet(CRNet):
    def __init__(self, img_dimwh=224, in_channels=3, downs_features=[32, 64, 128], atrous_rates=[6, 12, 18]):
        super(CRASPPNet, self).__init__(img_dimwh=img_dimwh, in_channels=in_channels, downs_features=downs_features)
        self.aspp = ASPP(
            in_channels=downs_features[-1],
            out_channels=downs_features[-1] * 2,
            atrous_rates=atrous_rates
        )
        self.bottom.linear = nn.Linear(self._cal_features__(img_dimwh) ** 2 * (downs_features[-1] * 2), 2)

    def forward(self, x):
        x = self.downs(x)
        x = self.aspp(x)
        x = self.bottom['globavgpool'](x)
        x = torch.flatten(x, 1)
        return self.bottom['linear'](x)

In [7]:
def test_craspp():
    x = torch.randn((16, 3, 224, 224))
    model = CRASPPNet()
    pred = model(x)
    print(pred.shape)

In [8]:
if __name__ == "__main__":
    test_aspp()
    test_craspp()

torch.Size([3, 256, 27, 27])
torch.Size([16, 2])
